# Reverse Correlation Analysis
<a id='revker'></a>

## Number of coherent dots per frame

Let's display the number of coherent dots on each frames, grouping by subject's choice.

In [ ]:
# count coherent dots on each frame
cohDots <- unique(DOTS[!is.na(validTrialCount),
                       .(numCohDots=sum(isCoherent), 
                       initDirection, 
                       endDirection, 
                       presenceCP, 
                       correct,
                       coherence,
                       viewingDuration,
                       choice),
                       by=.(frameIdx, validTrialCount)]) # I should not have to call unique() here... :(

# see if some frames have 0 coherent dots
# unique(cohDots[numCohDots == 0,.(validTrialCount)])

In [ ]:
# let's try to sign numCohDots according to the direction of motion (+ for R and - for L)
## add a copy of numCohDots column named signedNumCohDots, and an ID col that just indexes the rows
cohDots[,"signedNumCohDots" := .(numCohDots)]

In [ ]:
## on non-cp trials, adjust sign according to initDirection
cohDots[initDirection == "180" & !presenceCP, signedNumCohDots := -numCohDots] 

In [ ]:
## on cp-trials,
    # rough estimate of number of frames that are before the CP (for now)
if (length(unique(TRIALS$timeCP)) == 1) {
    foreFrameCount <- TRIALS[,timeCP][1] * FRAME_RATE_ESTIMATE
} 

In [ ]:
foreFrameCount

In [ ]:
rows <- cohDots[, .(rowNumber=.I[initDirection=="180" & presenceCP]), by=.(validTrialCount,frameIdx)]
cohDots[["signedNumCohDots"]][
    rows[frameIdx <= foreFrameCount,rowNumber]] <- -cohDots[["numCohDots"]][
        rows[frameIdx <= foreFrameCount,rowNumber]]

In [ ]:
rows <- cohDots[, .(rowNumber=.I[initDirection=="0" & presenceCP]), by=.(validTrialCount,frameIdx)]
cohDots[["signedNumCohDots"]][
    rows[frameIdx > foreFrameCount,rowNumber]] <- -cohDots[["numCohDots"]][
        rows[frameIdx > foreFrameCount,rowNumber]]

In [ ]:
# diagnostic
# unique(cohDots[presenceCP == TRUE,
#                .(initDirection,endDirection,presenceCP,numCohDots,signedNumCohDots) ,
#                by=.(frameIdx,validTrialCount)])

In [ ]:
# control output figure size
options(repr.plot.width=8, repr.plot.height=10)


# plot
ggplot(cohDots[validTrialCount > 466], aes(x=frameIdx, y=signedNumCohDots)) + 
    geom_point(aes(col=correct, shape=presenceCP), size=SMALL_DOT_SIZE) +
    facet_grid(validTrialCount~coherence) +
#    geom_line(aes(col=Event),size=.2) +
    geom_hline(yintercept = 0) +
    guides(colour = guide_legend(reverse=T), shape = guide_legend(reverse=T)) +
    labs(title="Signed # of Coherent Dots per Frames per Trials",
         subtitle=paste("From pilot",PILOT_NUMBER," dataset",sep=""),
         y="# Coherent Dots",
         x="Frame #",
         color="Correct", shape="Change Point") + 
    theme(plot.title=element_text(size=PLOT_TITLE_FONT_SIZE, lineheight=1.2),  # title
            plot.subtitle=element_text(size=PLOT_SUBTITLE_FONT_SIZE),  # subtitle
            plot.caption=element_text(size=PLOT_SUBTITLE_FONT_SIZE),                                 # caption
            axis.title.x=element_text(size=AXES_LABEL_FONT),  # X axis title
            axis.title.y=element_text(size=AXES_LABEL_FONT),  # Y axis title
            axis.text.x=element_text(size=AXES_FONT),  # X axis text
            axis.text.y=element_text(size=AXES_FONT))  # Y axis text

### Attempt at reverse kernels

What I will attempt here is the following:  
*display the average across trials, frame-by-frame, of the ME measure, conditioned on the subject's choice, and grouped by CP presence and viewing duration.*

In [ ]:
revKerNumCohDots <- cohDots[, .(ME=mean(signedNumCohDots), 
                                numTrials=.N,
                                sem=sd(signedNumCohDots) / sqrt(.N)), 
                            by=.(viewingDuration, presenceCP, frameIdx, choice)]

In [ ]:
head(revKerNumCohDots)

In [ ]:
revKerNumCohDots[,.(min(numTrials),max(numTrials))]

In [ ]:
str(cohDots)

In [ ]:
unique(revKerNumCohDots[numTrials < 5])[order(viewingDuration)]

In [ ]:
# control output figure size
options(repr.plot.width=8, repr.plot.height=14)

ggplot(revKerNumCohDots,aes(x=frameIdx,y=ME), colour=choice, group=choice) +
    geom_line(aes(col=choice), size=LINE_WIDTH) + 
    geom_ribbon(aes(ymin=ME-2*sem, ymax=ME+2*sem, col=choice), linetype=2, alpha=0.1) +
#     geom_errorbar(aes(ymin=ME-2*sem, ymax=ME+2*sem), colour="black", width=SMALL_ERROR_WIDTH)+
    facet_grid(viewingDuration~presenceCP) +
    guides(colour = guide_legend(reverse=T)) +
    labs(title="Left-aligned reverse kernels")

In [ ]:
print("Unfortunately, some CP trials have one more frame than non-CP trials:")
cohDots[order(viewingDuration, presenceCP),.(maxNumFrame=max(frameIdx)),by=.(viewingDuration, presenceCP)][viewingDuration>.2]

Not sure what I should do with these kernels. Should I trim one of their ends? Should I try to right-align them somehow?